In [1]:
from surprise import Dataset

In [3]:
# data.load_form_file , load_from_df
data = Dataset.load_builtin('ml-100k')
type(data)    #처음할때는 dataset에 있는 데이터 다운받게된다.

Dataset ml-100k could not be found. Do you want to download it? [Y/n] 

 y


Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to C:\Users\hoseo/.surprise_data/ml-100k


surprise.dataset.DatasetAutoFolds

In [5]:
print(len(data.raw_ratings))
data.raw_ratings[:10]

100000


[('196', '242', 3.0, '881250949'),
 ('186', '302', 3.0, '891717742'),
 ('22', '377', 1.0, '878887116'),
 ('244', '51', 2.0, '880606923'),
 ('166', '346', 1.0, '886397596'),
 ('298', '474', 4.0, '884182806'),
 ('115', '265', 2.0, '881171488'),
 ('253', '465', 5.0, '891628467'),
 ('305', '451', 3.0, '886324817'),
 ('6', '86', 3.0, '883603013')]

In [6]:
import pandas as pd

In [7]:
df = pd.DataFrame(data.raw_ratings, columns=['user', 'item','rate','id'])

In [9]:
print(df.shape)
df.head()

(100000, 4)


,user,item,rate,id
0,196,242,3.0,881250949
1,186,302,3.0,891717742
2,22,377,1.0,878887116
3,244,51,2.0,880606923
4,166,346,1.0,886397596


In [10]:
df.drop('id', axis=1, inplace=True)   #id 삭제

In [11]:
df.head()

,user,item,rate
0,196,242,3.0
1,186,302,3.0
2,22,377,1.0
3,244,51,2.0
4,166,346,1.0


In [14]:
df2=df.set_index(['user','item'])
df2.head()

,,rate
user,item,
196,242,3.0
186,302,3.0
22,377,1.0
244,51,2.0
166,346,1.0


In [17]:
# df3 = df2.unstack(fill_value=0)    # fill_value=0 : nan 을 0으로 채우기
df3 = df2.unstack()
df3.head()

rate                                               ...                    \
item    1   10  100 1000 1001 1002 1003 1004 1005 1006  ...  990 991 992  993   
user                                                    ...                     
1     5.0  3.0  5.0  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN NaN NaN  NaN   
10    4.0  NaN  5.0  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN NaN NaN  NaN   
100   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  3.0 NaN NaN  NaN   
101   3.0  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN NaN NaN  NaN   
102   3.0  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN NaN NaN  2.0   

                              
item 994 995 996 997 998 999  
user                          
1    NaN NaN NaN NaN NaN NaN  
10   NaN NaN NaN NaN NaN NaN  
100  NaN NaN NaN NaN NaN NaN  
101  NaN NaN NaN NaN NaN NaN  
102  NaN NaN NaN NaN NaN NaN  

[5 rows x 1682 columns]

In [18]:
df3.shape  #실제 사용자 943명, 영화는 1682개 

(943, 1682)

In [20]:
df3.iloc[200:205, 200:205].fillna('') 

rate                   
item 1179 118 1180 1181 1182
user                        
28                          
280         2         2    3
281                         
282                         
283

In [ ]:
# BaselineOnly  : 전체평균 - 사용자평균 = 가중치 조정값 적용  or 전체평균 - 제품평균 = 가중치 조정값 적용
# 'method':'als' = 사용자 가중치 조정값, 제품 가중치 조정값 둘다 만들어 진다.

In [21]:
from surprise.model_selection import KFold
from surprise import BaselineOnly, accuracy
import numpy as np

In [23]:
bsl_option = {'method':'als',       #als, sgd 두가지중 als가 좀더 최적화 모델이다. 
              'n_epochs':5,         #반복 횟수
              'reg_u':12,           #사용자 가중치
              'reg_i':5             #상품 가중치
             }

model = BaselineOnly(bsl_option)

np.random.seed(42)
accuracies = np.zeros(3)
cv=KFold(3)    
for i, (trainset, testset) in enumerate(cv.split(data)):
    model.fit(trainset)
    predictions = model.test(testset)
    accuracies[i] = accuracy.rmse(predictions, verbose=True)
    
accuracies.mean()

Estimating biases using als...
RMSE: 0.9419
Estimating biases using als...
RMSE: 0.9452
Estimating biases using als...
RMSE: 0.9446


0.9438942224301442

In [24]:
from surprise.model_selection import cross_validate

In [25]:
cross_validate(model, data, cv=5)

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...


{'test_rmse': array([0.93566615, 0.9417574 , 0.94765799, 0.94101401, 0.94324797]),
 'test_mae': array([0.74072242, 0.7456534 , 0.75064874, 0.74492599, 0.7473415 ]),
 'fit_time': (0.0937509536743164,
  0.12012529373168945,
  0.13756942749023438,
  0.11226797103881836,
  0.1273045539855957),
 'test_time': (0.10650753974914551,
  0.26561713218688965,
  0.10937356948852539,
  0.09374618530273438,
  0.11108565330505371)}

In [35]:
print(df3.shape)
print(df3.shape[0]*df3.shape[1])
print(df3.isnull().sum().sum())  # 비어있는 값이 많음.
print(df3.isnull().sum().sum()/(df3.shape[0]*df3.shape[1]))   #93%가 비어있는 값. 그래서 유사도 뽑기 어려움.

(943, 1682)
1586126
1486126
0.9369533063577546


In [26]:
from surprise import KNNBasic    #거리기반 유사도(비슷한 제품 찾는 도구)

In [30]:
# sim_options={'name':'msd'}  #오차 지정 방식: cosine, pearson ,  mse(msd) : 평균제곱차이 유사도
# sim_options={'name':'cosine'} 
sim_options={'name':'pearson'}

# model = KNNBasic(sim_options=sim_options)  
model = KNNBasic(k=10,min_k=3, sim_options=sim_options)  # k=이웃 갯수
cross_validate(model, data, cv=3)

#값들이 다 높아서 좋은게 아님....(비어있는 값이 너무 많음.)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.


{'test_rmse': array([1.05220479, 1.04395761, 1.05066796]),
 'test_mae': array([0.83122981, 0.82684673, 0.83265763]),
 'fit_time': (1.5815916061401367, 1.6264233589172363, 1.6203365325927734),
 'test_time': (3.4950501918792725, 3.659242630004883, 3.717651128768921)}

In [36]:
from surprise import SVD   #차원을 줄여주는 방식.

In [40]:
# svd = SVD()   #속성값 default=100
svd = SVD(n_factors=50) #속성 50개로 조정
svd

In [ ]:
# 속성이 많을때 selection(선택해서 이용) , extraxtion(새로운 속성 이용) 이용
# matrix_factorization :모든 사용자와 상품에 대해 다음 오차 함수를 최소화하는 요인 벡터를 찾아낸다

In [41]:
cross_validate(svd, data)

{'test_rmse': array([0.93665044, 0.93629514, 0.93026703, 0.93658739, 0.9303049 ]),
 'test_mae': array([0.73857802, 0.7360082 , 0.73335501, 0.74014327, 0.73376829]),
 'fit_time': (3.012519598007202,
  3.026353359222412,
  3.0479013919830322,
  3.042699098587036,
  3.093649387359619),
 'test_time': (0.35936450958251953,
  0.18749475479125977,
  0.18749332427978516,
  0.18749427795410156,
  0.18749427795410156)}

In [42]:
trainset = data.build_full_trainset() # 전체데이터를 훈련데이터로 사용

In [43]:
svd.fit(trainset)

In [44]:
testset = trainset.build_anti_testset() #평점이 없는 데이터 추출

In [45]:
print(type(testset))
print(len(testset))

<class 'list'>
1486126


In [46]:
predictions = svd.test(testset)  # 비어 있는 평점에 대한 예측

In [48]:
print(type(predictions))
print(len(predictions))
predictions[:10]

<class 'list'>
1486126


[Prediction(uid='196', iid='302', r_ui=3.52986, est=4.209303515333124, details={'was_impossible': False}),
 Prediction(uid='196', iid='377', r_ui=3.52986, est=2.8110420214480225, details={'was_impossible': False}),
 Prediction(uid='196', iid='51', r_ui=3.52986, est=3.4704631066660157, details={'was_impossible': False}),
 Prediction(uid='196', iid='346', r_ui=3.52986, est=3.8666105159866935, details={'was_impossible': False}),
 Prediction(uid='196', iid='474', r_ui=3.52986, est=4.142124186552839, details={'was_impossible': False}),
 Prediction(uid='196', iid='265', r_ui=3.52986, est=3.888709252942753, details={'was_impossible': False}),
 Prediction(uid='196', iid='465', r_ui=3.52986, est=3.5399196903759025, details={'was_impossible': False}),
 Prediction(uid='196', iid='451', r_ui=3.52986, est=3.7230962730421275, details={'was_impossible': False}),
 Prediction(uid='196', iid='86', r_ui=3.52986, est=3.8476681919505755, details={'was_impossible': False}),
 Prediction(uid='196', iid='1014'

In [52]:
from collections import defaultdict   # 키가 없으면 자동으로 생성

top_n = defaultdict(list)   #키가 없으면 리스트로 기본으로 만들어 달라는 의미
for uid, iid, _, est, _ in predictions:       # _ 부분은 r_ui(true_r), was_impossible 는 안불러 오겠다는 의미
    top_n[uid].append( (iid, est) )  # {사용자id: [(제품id, 평점)]} 으로 표시된다.


In [56]:
# top_n['196']
    
for uid, ratings in top_n.items():
    ratings.sort(key = lambda x: x[1], reverse=True)  # 사용자의 평점 정렬
    top_n[uid] = ratings[:10] # 평점이 높은 상위 10개의 예측 평점

In [57]:
for idx, (uid, ratings) in enumerate(top_n.items()):
    if idx ==10: break
    print(uid, [rating[0] for rating in ratings])

196 ['318', '64', '313', '480', '483', '408', '963', '50', '603', '169']
186 ['318', '513', '603', '272', '64', '511', '963', '316', '1194', '515']
22 ['408', '169', '56', '114', '357', '12', '483', '64', '178', '318']
244 ['134', '127', '408', '269', '483', '275', '12', '178', '285', '474']
166 ['169', '318', '190', '657', '483', '496', '1194', '963', '498', '651']
298 ['64', '190', '169', '408', '173', '963', '1039', '114', '513', '272']
115 ['408', '114', '483', '168', '513', '514', '182', '209', '251', '474']
253 ['169', '408', '114', '520', '174', '316', '498', '194', '272', '512']
305 ['513', '514', '9', '114', '211', '603', '515', '124', '659', '606']
6 ['179', '603', '657', '641', '661', '114', '654', '611', '615', '251']


In [ ]:
##  협력필터링

# 사용자 기법, item기법 => 유사도 (cosin, 거리(msd), corr)  
# => 유사도 만족하지 않을경우 SVD기법 이용하여 속성 줄여보자